In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load the Excel file to examine the data structure and verify column names
file_path = 'encerrados2024.xlsx'
data = pd.read_excel(file_path)

# Displaying the first few rows and column names to understand data layout
data.head(), data.columns


(      Caso                   Processo Status do Caso  \
 0   341385  0208030-51.2011.8.26.0100      Encerrado   
 1   344065  0108548-02.2008.8.06.0001      Encerrado   
 2   463826  0929225-60.2012.8.26.0506      Encerrado   
 3   501004  0010874-73.2012.8.13.0329      Encerrado   
 4  1065138  0002221-80.2013.8.26.0072      Encerrado   
 
                                                Ação  Área (LBCA)  \
 0  Conhecimento - Condenatória - Obrigação de Fazer     Área 150   
 1          Conhecimento - Condenatória - Dano Moral  140 - Cível   
 2  Conhecimento - Condenatória - Obrigação de Fazer     Área 150   
 3          Conhecimento - Condenatória - Dano Moral  140 - Cível   
 4          Conhecimento - Condenatória - Dano Moral  140 - Cível   
 
   Data de Cadastro Data da Citação Data da Distribuição Data de Encerramento  \
 0       2011-11-10      2011-11-03           2011-11-03           2024-10-01   
 1       2012-01-02      2009-04-01           2009-04-01           2024-05-14 

In [4]:
data.shape

(220, 60)

In [5]:
# Dropping rows where the 'Sentença - Dispositivo da Sentença' column has the value '***'
data_cleaned = data[data['Sentença - Dispositivo da Sentença'] != '***']

# Display the shape of the cleaned data to confirm rows have been removed
data_cleaned.shape

(214, 60)

In [7]:
# Evaluating the balance of the target variable 'Sentença - Dispositivo da Sentença'
target_counts = data_cleaned['Sentença - Dispositivo da Sentença'].value_counts(normalize=True) * 100
target_counts


Improcedente               27.102804
Extinção sem mérito        26.635514
Parcialmente Procedente    24.299065
Procedente                 21.962617
Name: Sentença - Dispositivo da Sentença, dtype: float64

In [9]:
# Explicitly creating a new variable with the specified mapping for 'Sentença - Dispositivo da Sentença'
mapping = {
    'Extinção sem mérito': 0,
    'Improcedente': 1,
    'Parcialmente Procedente': 2,
    'Procedente': 3
}

# Apply this mapping to create a new column with the specified numeric values
data_cleaned['target_custom_numeric'] = data_cleaned['Sentença - Dispositivo da Sentença'].map(mapping)

# Display the first few rows to confirm the new variable
data_cleaned[['Sentença - Dispositivo da Sentença', 'target_custom_numeric']].head()


C:\Users\crist\AppData\Local\Temp\ipykernel_23244\570199276.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['target_custom_numeric'] = data_cleaned['Sentença - Dispositivo da Sentença'].map(mapping)


,Sentença - Dispositivo da Sentença,target_custom_numeric
0,Improcedente,1
1,Improcedente,1
2,Procedente,3
3,Procedente,3
4,Parcialmente Procedente,2


In [ ]:
# Identifying date columns in the dataset by checking the data types
date_columns = data_cleaned.select_dtypes(include=['datetime64']).columns.tolist()




In [ ]:
# Redefining the feature set without attempting to drop columns that are not present in data_encoded
# Ensuring the correct columns are used for feature importance and PCA analysis

# Defining features and target
# Dropping identified date columns from the features
X = data_cleaned.drop(columns=['target_custom_numeric'] + ['Sentença - Dispositivo da Sentença'] + date_columns)
y = data_cleaned['target_custom_numeric']

# Using RandomForestRegressor to evaluate feature importance
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X, y)

# Extract feature importances and sort them
feature_importances = pd.Series(regressor.feature_importances_, index=X.columns).sort_values(ascending=False)
top_10_features_importance = feature_importances.head(10)

# PCA analysis to determine principal components
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# Get the explained variance ratio for each principal component
explained_variance_ratio = pca.explained_variance_ratio_

# Displaying top features based on feature importance and PCA explained variance ratio
top_10_features_importance, explained_variance_ratio

In [ ]:
# Train a RandomForestRegressor to evaluate feature importance without date columns
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X, y)

# Extract and sort feature importances
feature_importances = pd.Series(regressor.feature_importances_, index=X.columns).sort_values(ascending=False)
top_10_features_importance = feature_importances.head(10)

# Display the top 10 most important features influencing the target
top_10_features_importance


In [ ]:
data_cleaned.shape

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2

# Filter dataset for Department 'YouTube' in any of the columns AN to AR
data_youtube = data_cleaned[
    (data['Departamento 1'] == 'YouTube') |
    (data['Departamento 2'] == 'YouTube') |
    (data['Departamento 3'] == 'YouTube') |
    (data['Departamento 4'] == 'YouTube') |
    (data['Departamento 5'] == 'YouTube')
]

# Select columns for Causa Raiz 3 and target Sentença - Dispositivo da Sentença
data_filtered = data_youtube[['Causa Raiz 3', 'Sentença - Dispositivo da Sentença'] + list(data.columns[0:60])]

# Filter for only cases with a specific Causa Raiz 3 value if applicable
data_filtered = data_filtered.dropna(subset=['Causa Raiz 3', 'Sentença - Dispositivo da Sentença'])

# Encoding target and categorical variables for model processing
data_filtered['target'] = data_filtered['Sentença - Dispositivo da Sentença'].astype('category').cat.codes
data_filtered.drop(columns=['Sentença - Dispositivo da Sentença'], inplace=True)

# Encoding categorical features, filling missing values for simplicity
data_encoded = pd.get_dummies(data_filtered.fillna('Unknown'), drop_first=True)

# Split data into features (X) and target (y)
X = data_encoded.drop(columns=['target'])
y = data_encoded['target']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Select top features using SelectKBest with chi-squared test
selector = SelectKBest(chi2, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Identify selected feature names
selected_features = X.columns[selector.get_support(indices=True)].tolist()

# Train a RandomForestClassifier as a baseline model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_selected, y_train)

# Model evaluation
y_pred = clf.predict(X_test_selected)
report = classification_report(y_test, y_pred, output_dict=True)

# Output selected features and classification report
selected_features, report


C:\Users\crist\AppData\Local\Temp\ipykernel_23244\534380961.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_youtube = data_cleaned[


AttributeError: 'DataFrame' object has no attribute 'cat'